In [1]:
%pwd

'c:\\Users\\44787\\Desktop\\projects\\end-to-end-SMS-Spam-classifier\\research'

In [2]:
import os

os.chdir("../")
%pwd

'c:\\Users\\44787\\Desktop\\projects\\end-to-end-SMS-Spam-classifier'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [4]:
from src.SMSClassifier.constants import CONFIG_PATH
from src.SMSClassifier.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(self, config=CONFIG_PATH):
        self.config = read_yaml(config)

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path
        )
        return data_transformation_config

In [6]:
from src.SMSClassifier.logging import logger
import pandas as pd

import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from imblearn.over_sampling import RandomOverSampler

from collections import Counter

import joblib

In [7]:
class DataTransformation1:
    def __init__(self, config: DataTransformationConfig) -> None:
        self.config = config

    def clean_data(self):
        df = pd.read_csv(self.config.data_path, encoding='latin-1')
        # print(df.head(2))

        # remove unnecessary columns
        df = df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1)

        # rename columns name
        df = df.rename(columns={'v1': 'target', 'v2': 'msg'})

        # remove duplicates
        df = df.drop_duplicates(keep='first')

        # label encoding
        df.target = df.target.replace({'ham': 0, 'spam': 1})

        # text preprocessing
        df.msg = df.msg.apply(self.clean_msg)

        return df

    def clean_msg(self, msg, stemmer=PorterStemmer(), stop_words=set(stopwords.words('english'))):

        # remove html tags
        soup = BeautifulSoup(msg, 'html.parser')
        clean_text = soup.get_text()

        # convert to lower case and splits up the words
        words = word_tokenize(clean_text.lower())

        filter_words = []

        for word in words:
            # removing the stop words and punctuation
            if word not in stop_words and word.isalpha():
                filter_words.append(stemmer.stem(word))  # words stemming

        return ' '.join(filter_words)

    def tfidf_vectorizer(self, df):
        # tfidfvectorizer

        tfv = TfidfVectorizer(max_features=2500)
        X = tfv.fit_transform(df['msg']).toarray()
        y = df['target']
        
        df_new = pd.DataFrame(X, index=df.index)
        df_new['target'] = y
        
        logger.info(f'after tfidf data shape: {df_new.shape}')
        
        joblib.dump(tfv, os.path.join(self.config.root_dir, 'tfidfv.joblib'))
        
        return df_new


    def handle_imbalance(self, df):
        # balance the dataset
        X = df.drop('target', axis=1)
        y = df.target
        
        sampler = RandomOverSampler(random_state=42)
        X_sm, y_sm = sampler.fit_resample(X, y)

        logger.info('Resampled dataset shape %s' % Counter(y_sm))
        
        df_new = pd.DataFrame(X_sm, index=df.index)
        df_new['target'] = y_sm
        
        logger.info(df_new.shape)
        

        return df_new

    def train_test_splits(self, df):
        train, test = train_test_split(df)
        train.to_csv(os.path.join(
            self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(
            self.config.root_dir, "test.csv"), index=False)
        logger.info("Splited data into training and test sets")
        logger.info(f'train shape: {train.shape}')
        logger.info(f'test shape: {test.shape}')

In [8]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation1(data_transformation_config)
    df = data_transformation.clean_data()
    df = data_transformation.tfidf_vectorizer(df)
    data_transformation.train_test_splits(df)
except Exception as e:
    raise e

[2024-02-06 15:56:03,622: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-06 15:56:03,628: INFO: common: created directory at: artifacts/data_transformation]


C:\Users\44787\AppData\Local\Temp\ipykernel_19564\1269566412.py:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.target = df.target.replace({'ham': 0, 'spam': 1})
C:\Users\44787\AppData\Local\Temp\ipykernel_19564\1269566412.py:29: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(msg, 'html.parser')


[2024-02-06 15:56:06,797: INFO: 1269566412: after tfidf data shape: (5169, 2501)]
[2024-02-06 15:56:15,360: INFO: 1269566412: Splited data into training and test sets]
[2024-02-06 15:56:15,361: INFO: 1269566412: train shape: (3876, 2501)]
[2024-02-06 15:56:15,362: INFO: 1269566412: test shape: (1293, 2501)]
